In [1]:
#importing required modules
import tweepy
import pandas as pd
import numpy as np
import re

In [2]:
#code to upload file which consists of twitter API keys
from google.colab import files
uploaded=files.upload()


Saving Login.csv to Login.csv


In [3]:
#creating a dataframe for API keys
data=pd.read_csv("Login.csv")

In [4]:
#gettings the keys from dataframe for authentication
consumerKey=data['Key'][0]
consumerSecret=data['Key'][1]
accessToken=data['Key'][2]
accessTokenSecret=data['Key'][3]

In [5]:
#Getting the API Authentication
authenticate=tweepy.OAuthHandler(consumerKey,consumerSecret)
api=tweepy.API(authenticate,wait_on_rate_limit=True)


In [6]:
#getting the tweet posts from API using search query and language
posts=api.user_timeline(screen_name="jamescharles", count=50000, lang="en", tweet_mode="extended")

In [7]:
#Printing the first 5 tweets
i=1
for tweets in posts[:5]:
    print(str(i)+tweets.full_text+"\n")
    i+=1

1RT @jamescharles: NEW VIDEO IS FINALLY LIVE!

RECREATING MY FOLLOWERS HALLOWEEN MAKEUP! 💀🔪 https://t.co/Tt3k9xIEvM https://t.co/q8HoH27rUA

2NEW VIDEO IS FINALLY LIVE!

RECREATING MY FOLLOWERS HALLOWEEN MAKEUP! 💀🔪 https://t.co/Tt3k9xIEvM https://t.co/q8HoH27rUA

3RT @jamescharles: NEW VIDEO IS OUT NOW!

I SET MY MAKEUP ROUTINE ON FIRE! 🔥 This challenge was insane lmfaooo enjoy! https://t.co/T5U2WVh9a…

4@mrjamiecharlie HAPPY BIRTHDAY ELL I LOVEEEE U

5RT @patrickstarrr: JAMES OMG THIS IS CRAZY LMAO!! “@OneSizeBeauty is fireproof” I screamed 😭😂❤️



In [8]:
#converting the tweets data as dataframe
df=pd.DataFrame([tweet.full_text for tweet in posts],columns=['Tweets'])
df.head()

,Tweets
0,RT @jamescharles: NEW VIDEO IS FINALLY LIVE!\n...
1,NEW VIDEO IS FINALLY LIVE!\n\nRECREATING MY FO...
2,RT @jamescharles: NEW VIDEO IS OUT NOW!\n\nI S...
3,@mrjamiecharlie HAPPY BIRTHDAY ELL I LOVEEEE U
4,RT @patrickstarrr: JAMES OMG THIS IS CRAZY LMA...


In [9]:
#Removing links, additional white spaces, Hashtags and RT tags, @Mentionings
def cleanTxt(text):
  text=re.sub(r'@[A-Za-z0-9]+','',text)
  text=re.sub(r'#','',text)
  text=re.sub(r'RT[\s]+','',text)
  text=re.sub(r'https?:\/\/\S+','',text)
  text=re.sub(r':','',text)
  return text
df['Tweets']=df['Tweets'].apply(cleanTxt)
df

,Tweets
0,NEW VIDEO IS FINALLY LIVE!\n\nRECREATING MY F...
1,NEW VIDEO IS FINALLY LIVE!\n\nRECREATING MY FO...
2,NEW VIDEO IS OUT NOW!\n\nI SET MY MAKEUP ROUT...
3,HAPPY BIRTHDAY ELL I LOVEEEE U
4,JAMES OMG THIS IS CRAZY LMAO!! “ is fireproof...
5,"he is, agreed, but I do my thumbnails myself!..."
6,today’s video thumbnail before and after 🤓
7,NEW VIDEO IS OUT NOW!\n\nI SET MY MAKEUP ROUTI...
8,NEW VIDEO IS LIVE!\nI TRIED POLE DANCING FOR ...
9,NEW VIDEO IS LIVE!\nI TRIED POLE DANCING FOR T...


In [10]:
#removing punctuations
df['Tweets'] = df['Tweets'].str.replace('[^\w\s]','') 
df['Tweets'] =df['Tweets'].str.replace('\n',' ') 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [11]:
df

,Tweets
0,NEW VIDEO IS FINALLY LIVE RECREATING MY FOLL...
1,NEW VIDEO IS FINALLY LIVE RECREATING MY FOLLO...
2,NEW VIDEO IS OUT NOW I SET MY MAKEUP ROUTINE...
3,HAPPY BIRTHDAY ELL I LOVEEEE U
4,JAMES OMG THIS IS CRAZY LMAO is fireproof I ...
5,he is agreed but I do my thumbnails myself lol
6,todays video thumbnail before and after
7,NEW VIDEO IS OUT NOW I SET MY MAKEUP ROUTINE ...
8,NEW VIDEO IS LIVE I TRIED POLE DANCING FOR TH...
9,NEW VIDEO IS LIVE I TRIED POLE DANCING FOR THE...


In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
#tokenizing the words and sentences
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk import punkt
def tokenize(data, return_sentences = True):
    sentences = data.split('\r\n')
    sentences = [sent.strip() for sent in sentences]
    sentences = [sent for sent in sentences if len(sent) > 0]

    word_tokens = [word_tokenize(sent.lower()) for sent in sentences]
    if return_sentences:
        return sentences, word_tokens
    return word_tokens

In [19]:
#call tokenize() to get sentences and words
data=df.to_string(index=False)
sentences, word_tokens = tokenize(data)

In [41]:
len(word_tokens[0])

265

In [51]:
#method to perform train test split
train_data=word_tokens[0][:240]
test_data=word_tokens[0][240:]


In [52]:
#count words
from collections import Counter

def count_words(data):
    cnt = Counter()
    for words in data:
        cnt.update(words)
    return cnt

In [53]:
word_count = count_words(train_data)


In [54]:
#create vocabulary
def create_vocabulary(word_count, min_frequency=0, max_words=0):
    if min_frequency > 0:
        vocabulary = []
        for word, count in word_count.items():
            if count >= min_frequency:
                vocabulary.append(word)
    if max_words > 0:
        vocabulary = [word[0] for word in word_count.most_common(max_words)]

    return vocabulary

In [55]:
#repalce words with unknown tokens
def handle_unk_words(data, vocabulary, unk_token="<unk>"):
    handled_data = []
    for sentences in data:
        sentence = []
        for word in sentences:
            if word in vocabulary:
                sentence.append(word)
            else:
                sentence.append(unk_token)
        handled_data.append(sentence)
    return handled_data

In [56]:
vocabulary = create_vocabulary(word_count, min_frequency=3)
preprocessed_data = handle_unk_words(train_data, vocabulary=vocabulary)

In [59]:
preprocessed_data_test = handle_unk_words(test_data, vocabulary=vocabulary)


In [60]:
#n-gram model
from collections import defaultdict
def count_n_grams(data, n, start_token="<s>", end_token="<e>"):
    n_grams = defaultdict(int)
    for sentence in data:
        sentence = [start_token]*n + sentence + [end_token]
        m = len(sentence) if n == 1 else len(sentence)-1
        for i in range(m):
            n_gram = sentence[i:i+n]
            n_gram = tuple(n_gram)
            n_grams[n_gram] += 1
    return n_grams


In [61]:
#calculate probability
def calculate_probability(word, previous_n_gram, n_gram_counts, n_plusone_gram_counts,vocabulary_size,k=1):
    n_plus_one_gram = tuple(previous_n_gram + [word])
    n_plus_one_gram_count = n_plusone_gram_counts[n_plus_one_gram] \
                    if n_plus_one_gram in n_plusone_gram_counts else 0
    numerator = n_plus_one_gram_count + k

    previous_n_gram = tuple(previous_n_gram)
    previous_n_gram_count = n_gram_counts[previous_n_gram] \
                    if previous_n_gram in n_gram_counts else 0

    denominator = previous_n_gram_count + k * vocabulary_size


    probability = numerator / denominator
    return probability

In [62]:
#evaluating model using perplexity
def perplexity(sentence,n_gram_counts, 
               n_plusone_gram_counts, 
               vocabulary, k=1.0, log_perplexity=True):
    # find n as in n-grams
    n = len(list(n_gram_counts.keys())[0])

    sentence = ["<s>"] * n + sentence + ["<e>"]
    sentence = tuple(sentence)

    vocabulary_size = len(vocabulary)

    # N = len(sentence) if n == 1 else len(sentence)-1
    N = len(sentence)

    if log_perplexity:
        temp_sum = 0.0
    else:
        temp_product = 1.0

    for t in range(n,N):
        n_gram = sentence[t-n:t] 
        # Because the loop starts at n i.e. it excludes first n words as
        # part of n-gram where t is current word so n-gram will be previous n
        # words

        word = sentence[t]

        probability = calculate_probability(word, n_gram, 
                                            n_gram_counts,
                                            n_plusone_gram_counts,
                                            vocabulary, k=k)
        if log_perplexity:
            temp_sum += (1 / probability)
        else:
            temp_product *= (1 / probability)
        
    if log_perplexity:
        perplexity = - (1/N) * temp_sum
    else:
      perplexity = temp_product**(1/N)

    return perplexity

In [63]:
#calcualte probabilities
def calculate_probabilities(previous_n_gram, 
                            n_gram_counts, n_plusone_gram_counts, 
                            vocabulary, k=1.0):
    probabilities = dict()
    vocabulary = vocabulary + ["<e>","<unk>"]
    vocabulary_size = len(vocabulary)

    for word in vocabulary:
        probability = calculate_probability(word,
                                            previous_n_gram, 
                                            n_gram_counts, 
                                            n_plusone_gram_counts, 
                                            vocabulary_size,
                                            k=1.0)
        probabilities[word] = probability

    return probabilities

In [64]:
#suggest words
def suggest_a_word(previous_tokens, 
                   n_gram_counts, 
                   n_plusone_gram_counts, 
                   vocabulary, 
                   k=1.0,
                   start_with=None):
    
    n = len(list(n_gram_counts.keys())[0])

    previous_n_gram = previous_tokens[-n:] 

    probabilities = calculate_probabilities(previous_n_gram,
                                            n_gram_counts,
                                            n_plusone_gram_counts,
                                            vocabulary,
                                            k=k)
    max_probability = 0
    suggestion = None

    for word, prob in probabilities.items():
        if start_with:
            if not word.startswith(start_with):
                continue
        
        if prob > max_probability:
            max_probability = prob
            suggestion = word
    return (suggestion, max_probability)

In [65]:
#get suggestions
def get_suggestions(previous_tokens, 
                    n_gram_counts, 
                    n_plusone_gram_counts,
                    vocabulary,
                    k=1.0,
                    n_suggestions=4,
                    starts_with=None):
    
    n = len(list(n_gram_counts.keys())[0])

    previous_n_gram = previous_tokens[-n:]

    probabilities = calculate_probabilities(
        previous_n_gram,
        n_gram_counts,
        n_plusone_gram_counts,
        vocabulary,
        k=k
    )

    sorted_probabilities = {
        word : probability for word, probability in \
        sorted(probabilities.items(), key=lambda item: item[1], reverse=True)
            }
    if starts_with:
        suggestions = dict()
        for word, prob in sorted_probabilities.items():
            if word.startswith(starts_with):
                suggestions[word] = prob
            else:
                continue
        return list(suggestions.items())[:n_suggestions]

    return list(sorted_probabilities.items())[:n_suggestions]

In [68]:
#test data
sentences=[['New','Video','Out','Now','Check','it','out']]
unique_words = list(set(sentences[0]))
unigram_counts = count_n_grams(sentences, 1)
bigram_counts = count_n_grams(sentences, 2)
trigram_counts=count_n_grams(sentences,3)
fourgram_counts=count_n_grams(sentences,4)
fivegram_counts=count_n_grams(sentences,5)

previous_tokens = ["New", "Video"]
tmp_suggest1 = suggest_a_word(previous_tokens, unigram_counts, bigram_counts, unique_words, k=1.0)
print(f"The previous words are 'New Video',\n\tand the suggested word is `{tmp_suggest1[0]}` with a probability of {tmp_suggest1[1]:.4f}")



The previous words are 'New Video',
	and the suggested word is `Out` with a probability of 0.2000


In [69]:
get_suggestions(previous_tokens,unigram_counts,bigram_counts,unique_words,k=1,n_suggestions=4,starts_with="O")

[('Out', 0.2)]

In [72]:
previous_tokens = previous_tokens + ['The']
get_suggestions(previous_tokens, fourgram_counts, fivegram_counts, unique_words, k=1.0)


[('Check', 0.1111111111111111),
 ('it', 0.1111111111111111),
 ('Out', 0.1111111111111111),
 ('Video', 0.1111111111111111)]